# IAM Tractometry with [TractSeg](http://yeatmanlab.github.io/pyAFQ/index.html)

This script ustilizes TractSeg to perform tractometry on select IAM subjects. This scripts serves as an investigational tool in determining a DKI post-processing pipeline.

## Load all required modules

Load all required Python modules

In [5]:
import os
import os.path as op
import glob
import tscompatibility as ts
import subprocess
from IPython.display import Image

## Define paths
Define paths to various input/output folders. Input folder needs to be defined as that containing subject folders, while the output directory will contain TractSeg's outputs.

In [6]:
input = '/Users/dataprocessing/Documents/IAM/TractSeg_Subs_BAK'
output = '/Users/dataprocessing/Documents/IAM/TractSeg'
path_mni = '/Users/dataprocessing/Documents/IAM/TractSeg/MNI_FA_template.nii.gz'

## 1. Read Subject Information

Read the content of input directory to form a list of subjects.

In [7]:
subPaths = glob.glob(op.join(input, '*'))

## 2. Process Each Subject

In [4]:
failedRun = []
for i in subPaths:
    subID = op.basename(i)
    
    subOut = op.join(output, subID)
    ts.runtractseg(input=i,
               output=subOut,
               template=path_mni,
               docker=None)
#     try:
#         runtractseg(input=i,
#                    output=subOut)
#     except:
#         failedRun.append(subID)

Processing IAM_1158

----- Files being processed -----
FA: /Users/dataprocessing/Documents/IAM/TractSeg_Subs_BAK/IAM_1158/metrics/fa.nii
DWI: /Users/dataprocessing/Documents/IAM/TractSeg_Subs_BAK/IAM_1158/dwi_preprocessed.nii
BVEC: /Users/dataprocessing/Documents/IAM/TractSeg_Subs_BAK/IAM_1158/dwi_preprocessed.bvec
BVAL: /Users/dataprocessing/Documents/IAM/TractSeg_Subs_BAK/IAM_1158/dwi_preprocessed.bval
Mask: /Users/dataprocessing/Documents/IAM/TractSeg_Subs_BAK/IAM_1158/brain_mask.nii
---------------------------------

STAGE 1: Image registration into MNI space

Removing NaNs from scalar image
Computing transformation affine matrix


KeyboardInterrupt: 

In [ ]:
print(failedRun)

In [ ]:
subList = [op.basename(x) for x in subPaths]
for i in subList:
    print(i)

## Run group analysis

In [ ]:
print('STAGE 3: Group analysis')
print('')
arg = ['plot_tractometry_results']
arg.extend(['-i', '/Users/dataprocessing/Documents/IAM/TractSeg/subjects_FA.txt'])
arg.extend(['-o', '/Users/dataprocessing/Documents/IAM/TractSeg/tractometry_result_FA.png'])
# arg.extend(['--plot3D', 'metric', '--tracking_format', 'trk'])
arg.append('--mc')
print('COMMAND: {}'.format(' '.join(arg)))
completion = subprocess.run(arg)
if completion.returncode != 0:
    raise Exception('Unable to run TractSeg: Group analysis. '
                    'Check above for errors')

In [ ]:
# View plot
Image(filename='/Users/dataprocessing/Documents/IAM/TractSeg/tractometry_result.png') 

## Run for MK

In [8]:
for i in subPaths:
    subID = op.basename(i)
    subOut = op.join(output, subID)
    
    mk_path = op.join(i, 'metrics', 'mk.nii')
    mk_mni_path = op.join(subOut, 'MK_MNI.nii.gz')
    mk_nan_path = op.join(subOut, 'MK_NAN.nii.gz')
    mk_zero_path = op.join(subOut, 'MK_ZERO.nii.gz')
    omat_path = op.join(subOut, 'FA_2_MNI.mat')
    
    tracking_dir = op.join(subOut, 'TOM_trackings')
    end_dir = op.join(subOut, 'endings_segmentations')
    
    
    ts.nan_to_zero(mk_path, mk_nan_path)
    # Remove negative values from MK
    ts.threshold(mk_nan_path, mk_zero_path, [0, 2])
    
    ts.applyTransform(
        moving=mk_zero_path,
        template=path_mni,
        omat=omat_path,
        out=mk_mni_path,
    )
    # Remove obsolete files
    os.remove(mk_nan_path)
    os.remove(mk_zero_path)
    
    ts.segTractometry(
        metric=mk_mni_path,
        tracking_dir=tracking_dir,
        end_dir=end_dir,
        out=op.join(subOut, 'Tractometry_MK.csv')
    )

In [9]:
arg = ['plot_tractometry_results']
arg.extend(['-i', '/Users/dataprocessing/Documents/IAM/TractSeg/subjects_MK.txt'])
arg.extend(['-o', '/Users/dataprocessing/Documents/IAM/TractSeg/tractometry_result_MK.png'])
# arg.extend(['--plot3D', 'metric', '--tracking_format', 'trk'])
arg.append('--mc')
completion = subprocess.run(arg)
if completion.returncode != 0:
    raise Exception('Unable to run TractSeg: Group analysis. '
                    'Check above for errors')

## Run for KFA

In [11]:
for i in subPaths:
    subID = op.basename(i)
    subOut = op.join(output, subID)
    
    mk_path = op.join(i, 'metrics', 'kfa.nii')
    mk_mni_path = op.join(subOut, 'KFA_MNI.nii.gz')
    mk_nan_path = op.join(subOut, 'KFA_NAN.nii.gz')
    mk_zero_path = op.join(subOut, 'KFA_ZERO.nii.gz')
    omat_path = op.join(subOut, 'FA_2_MNI.mat')
    
    tracking_dir = op.join(subOut, 'TOM_trackings')
    end_dir = op.join(subOut, 'endings_segmentations')
    
    
    ts.nan_to_zero(mk_path, mk_nan_path)
    # Remove negative values from MK
    
    ts.applyTransform(
        moving=mk_nan_path,
        template=path_mni,
        omat=omat_path,
        out=mk_mni_path,
    )
    ts.threshold(mk_mni_path, mk_zero_path, [0, 1])
    # Remove obsolete files
    os.remove(mk_nan_path)
    os.remove(mk_mni_path)
    os.rename(mk_zero_path, mk_mni_path)
    
    ts.segTractometry(
        metric=mk_mni_path,
        tracking_dir=tracking_dir,
        end_dir=end_dir,
        out=op.join(subOut, 'Tractometry_KFA.csv')
    )

In [14]:
arg = ['plot_tractometry_results']
arg.extend(['-i', '/Users/dataprocessing/Documents/IAM/TractSeg/subjects_KFA.txt'])
arg.extend(['-o', '/Users/dataprocessing/Documents/IAM/TractSeg/tractometry_result_KFA.png'])
# arg.extend(['--plot3D', 'metric', '--tracking_format', 'trk'])
arg.append('--mc')
completion = subprocess.run(arg)
if completion.returncode != 0:
    raise Exception('Unable to run TractSeg: Group analysis. '
                    'Check above for errors')